In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [ ]:
weather_df = pd.read_csv('../output_data/cities.csv')
#weather_df.drop(weather_df['Unnamed: 0'])
weather_df.head()

In [ ]:
# Configure maps
gmaps.configure(api_key=g_key)

In [ ]:
humid_df = weather_df[['Lat','Lng','Humidity']]
humid_df.head()

In [ ]:
locations = humid_df[['Lat','Lng']]
humidity = humid_df['Humidity']

In [ ]:
fig = gmaps.figure(zoom_level =2, center = (15,15))

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, max_intensity = 100, point_radius = 2)

fig.add_layer(heat_layer)


fig

In [ ]:
# Narrow down the DataFrame to fit weather conditions
ideal_df = weather_df.loc[(weather_df['MaxTemp'] < 85) & (weather_df['MaxTemp'] > 77)]
ideal_df = ideal_df.loc[weather_df['WindSpeed'] < 10]
hotel_df = ideal_df.loc[weather_df['Cloudiness']< 10]
hotel_df

In [ ]:
# Add hotel column
hotel_df['Hotel Name'] = ""
hotel_df = hotel_df[['City','Lat','Lng','Hotel Name','MaxTemp','Humidity','Cloudiness','WindSpeed','Country','Date']]
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
my_params = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
   
    lat = row['Lat']
    lng = row['Lng']
        
    my_params['location'] = f'{lat},{lng}'
    
    # assemble url and make API request
    print(f"Retrieving Hotel for Index {index}.")
    response = requests.get(base_url, params=my_params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=hotel_info)

fig = gmaps.figure(zoom_level =2, center = (15,15))

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig